In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 113.0.5672
Get LATEST chromedriver version for 113.0.5672 google-chrome
There is no [win32] chromedriver for browser 113.0.5672 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\liz14\.wdm\drivers\chromedriver\win32\113.0.5672.63]


In [2]:
url = "https://www.imdb.com/list/ls006444168/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# Webscrape imdb movies from books list. 
movies_list = soup.find_all('div', class_='lister-list')

# Pull movie title, year released, movie duration, genre, movie rating ("certificate"), star rating, gross amount.

title = movies_list[0].find('h3').find('a').text

year = movies_list[0].find('span',class_='lister-item-year').text

movie_length = movies_list[0].find('span',class_='runtime').text

genre = movies_list[0].find('span',class_='genre').text

# javascript for certificate: # <span class="certificate">R</span>
certificate = movies_list[0].find('span',class_='certificate').text

# javascript for rating: <span class="ipl-rating-star__rating">7.5</span>
stars = movies_list[0].find('span',class_='ipl-rating-star__rating').text

# javascript for movie gross:
# <span class="text-muted">Gross:</span>
# <span name="nv" data-value="18,335,230">$18.34M</span>

# chatGPT   
gross = movies_list[0].find('span', string='Gross:')
gross_value = gross.find_next_sibling('span')['data-value']

#********************Discard later?
# # find the <p> tag with class 'text_muted text-small'
##p_tag = soup.find('p', class_='text-muted text-small')
#p_tag = soup.find('span', class_='text-muted').text
#p_tag = soup.find('span', name_='nv')


# # find the <span> tag with name 'nv' inside the <p> tag
#span_tag = p_tag.find('span', name='nv')

# # extract the text content of the <span> tag.
# text = span_tag.text.strip()

In [4]:
certificate

'R'

In [5]:
# Pull movie title, year released, movie duration, genre, movie rating ("certificate"), star rating, gross amount.

# Create dictionary of lists to hold....

movies_fr_books_dict = {}
movie_title =[]
year = []
movie_length =[]
movie_genre=[]
movie_rating=[]
star_rating=[]
gross_amount=[]

def get_movietitle():
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    movies_list = soup.find_all('div', class_='lister-item-content')
    
    for movie in movies_list:
        title = movie.find('h3').find('a').text
        
        movie_year = movie.find('span', class_='lister-item-year').text
        
        movie_len = movie.find('span', class_='runtime').text
        
        genre = movie.find('span', class_='genre').text.strip()
        
        try:
            certificate = movie.find('span', class_='certificate').text
        except AttributeError:
            certificate = 'N/A'
        
        stars = movie.find('span', class_='ipl-rating-star__rating').text
        
        gross = movies_list[0].find('span', string='Gross:')
        gross_value = gross.find_next_sibling('span')['data-value']

        # recording values
        movie_title.append(title)
        year.append(movie_year)
        movie_length.append(movie_len)
        movie_genre.append(genre)        
        movie_rating.append(certificate)
        star_rating.append(stars)
        gross_amount.append(gross_value)

get_movietitle()

In [6]:
genre

'\nBiography, Drama            '

In [7]:
movies_fr_books_dict['movie_title'] = movie_title
movies_fr_books_dict['year'] = year
movies_fr_books_dict['movie_length'] = movie_length
movies_fr_books_dict['movie_genre'] = movie_genre
movies_fr_books_dict['movie_rating'] = movie_rating
movies_fr_books_dict['star_rating'] = star_rating
movies_fr_books_dict['gross_amount'] = gross_value

In [8]:
import pandas as pd
moviesfrombooks_data = pd.DataFrame(movies_fr_books_dict)
moviesfrombooks_data.head(5)

movie_title    year movie_length  \
0                        127 Hours  (2010)       94 min   
1                       100 Rifles  (1969)      110 min   
2                 The Hunger Games  (2012)      142 min   
3  The Hunger Games: Catching Fire  (2013)      146 min   
4                 A Beautiful Mind  (2001)      135 min   

                 movie_genre movie_rating star_rating gross_amount  
0           Biography, Drama            R         7.5   18,335,230  
1      Adventure, Drama, War            R           6   18,335,230  
2  Action, Adventure, Sci-Fi        PG-13         7.2   18,335,230  
3  Action, Adventure, Sci-Fi        PG-13         7.5   18,335,230  
4           Biography, Drama        PG-13         8.2   18,335,230

In [9]:
#Identify datatype.
moviesfrombooks_data.dtypes

movie_title     object
year            object
movie_length    object
movie_genre     object
movie_rating    object
star_rating     object
gross_amount    object
dtype: object

In [10]:
# Display random sample of 10 rows from dataframe, moviesfrombooks_data

moviesfrombooks_data.sample(n=10)

movie_title        year movie_length  \
94                      The Soloist      (2009)      117 min   
22  The Girl with the Dragon Tattoo      (2011)      158 min   
92                    Up in the Air  (I) (2009)      109 min   
49            Sense and Sensibility      (1995)      136 min   
65                The Pelican Brief      (1993)      141 min   
14              Message in a Bottle      (1999)      126 min   
83                     Tamara Drewe      (2010)      107 min   
8                  A Little Romance      (1979)      110 min   
42               My Sister's Keeper      (2009)      109 min   
39                          One Day      (2011)      107 min   

                   movie_genre movie_rating star_rating gross_amount  
94     Biography, Drama, Music        PG-13         6.7   18,335,230  
22       Crime, Drama, Mystery            R         7.8   18,335,230  
92      Comedy, Drama, Romance            R         7.4   18,335,230  
49              Drama, Romance           PG         7.7   18,335,230  
65       Crime, Drama, Mystery        PG-13         6.6   18,335,230  
14              Drama, Romance        PG-13         6.2   18,335,230  
83      Comedy, Drama, Romance            R         6.2   18,335,230  
8   Adventure, Comedy, Romance           PG         7.4   18,335,230  
42               Drama, Family        PG-13         7.3   18,335,230  
39              Drama, Romance        PG-13           7   18,335,230

In [11]:
moviesfrombooks_data.iloc[0]['movie_genre'].split(', ')

['Biography', 'Drama']

In [12]:
# extract digits only from `year` column
moviesfrombooks_data['year'] = moviesfrombooks_data['year'].str.extract(r'\((\d+)\)')

In [13]:
# extract digits only from `movie_length` column
moviesfrombooks_data['movie_length'] = moviesfrombooks_data['movie_length'].str.extract(r'(\d+)\s*min')

In [14]:
# split genre text string into a list of genres
moviesfrombooks_data['movie_genre'] = moviesfrombooks_data['movie_genre'].str.split(', ')

In [15]:
moviesfrombooks_data = pd.get_dummies(moviesfrombooks_data, columns = ['movie_rating'])

In [16]:
# pd.get_dummies(moviesfrombooks_data, columns=['movie_genre']) DOES NOT WORK

# adpated from stack overflow
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

moviesfrombooks_data = moviesfrombooks_data.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(moviesfrombooks_data['movie_genre']), #.pop if you want to remove original column of list
                index=moviesfrombooks_data.index,
                columns=mlb.classes_))

In [17]:
moviesfrombooks_data.head()

movie_title  year movie_length  \
0                        127 Hours  2010           94   
1                       100 Rifles  1969          110   
2                 The Hunger Games  2012          142   
3  The Hunger Games: Catching Fire  2013          146   
4                 A Beautiful Mind  2001          135   

                   movie_genre star_rating gross_amount  movie_rating_N/A  \
0           [Biography, Drama]         7.5   18,335,230                 0   
1      [Adventure, Drama, War]           6   18,335,230                 0   
2  [Action, Adventure, Sci-Fi]         7.2   18,335,230                 0   
3  [Action, Adventure, Sci-Fi]         7.5   18,335,230                 0   
4           [Biography, Drama]         8.2   18,335,230                 0   

   movie_rating_Not Rated  movie_rating_PG  movie_rating_PG-13  ...  Horror  \
0                       0                0                   0  ...       0   
1                       0                0                   0  ...       0   
2                       0                0                   1  ...       0   
3                       0                0                   1  ...       0   
4                       0                0                   1  ...       0   

   Music  Musical  Mystery  Romance  Sci-Fi  Sport  Thriller  War  Western  
0      0        0        0        0       0      0         0    0        0  
1      0        0        0        0       0      0         0    1        0  
2      0        0        0        0       1      0         0    0        0  
3      0        0        0        0       1      0         0    0        0  
4      0        0        0        0       0      0         0    0        0  

[5 rows x 32 columns]

In [18]:
# Save/export dataframe, moviesfrombooks_data, to csv file.

moviesfrombooks_data.to_csv('Resources/imdb_movies_from_books.csv', index=True)

In [19]:
browser.quit()